In [41]:
from pathlib import Path

import numpy as np
from numba import njit, prange

import pandas as pd

In [42]:

def Softmax(x: np.ndarray) -> float:
    y = np.exp(x)
    return y/np.sum(y)


@njit(fastmath=True)
def Softmax_jit(x: np.ndarray) -> float:
    y = np.exp(x)
    return y/np.sum(y)


@njit(fastmath=True)
def grads(xBatch: np.ndarray, yBatch: np.ndarray, w: np.ndarray, b: np.ndarray) -> tuple[np.ndarray]:
    n = xBatch.shape[0]

    dw = np.zeros(w.shape)
    db = np.zeros(b.shape)
    
    for i in prange(n):
        y = Softmax_jit(xBatch[i] @ w + b)
        dw += np.atleast_2d(xBatch[i]).T * (np.sum(yBatch[i]) * y - yBatch[i])
        db += np.sum(yBatch[i]) * y - yBatch[i]
    
    return (dw, db)

class Perceptron:
    def __init__(self, nIn: int, nOut: int) -> None:
        self.nIn = nIn
        self.nOut = nOut
        self.w: np.ndarray = np.random.uniform(-1, 1, (nIn, nOut))
        self.b: np.ndarray = np.zeros((nOut))

    def predict(self, x:np.ndarray) -> np.ndarray:
        return Softmax(x @ self.w + self.b)

    def train(self, xTrain: np.ndarray, yTrain: np.ndarray, lr, batch_size, max_iter) -> None:
        n = xTrain.shape[0]

        for i in range(max_iter):
            idxs = np.random.choice(a=np.arange(n), size=batch_size, replace=False)

            dw, db = grads(xTrain[idxs], yTrain[idxs], self.w, self.b)
            
            self.w -= lr*dw
            self.b -= lr*db

    def loss(self, x: np.ndarray, y: np.ndarray) -> float:        
        Ypred = np.array([self.predict(xi) for xi in x])
        h = - 1/self.nOut * np.array(y*np.log(Ypred))
        return 1/y.shape[0] * np.sum(h)

In [43]:
df = pd.read_csv(Path('..', '..', 'data', 'iris_csv.csv'))

for c in df.columns[0:4]:
    df[c] = (df[c]-df[c].mean())/df[c].std()

df['synth1'] = df['petallength']*df['petalwidth']
df['synth2'] = df['sepallength']*df['petallength']
df['synth3'] = df['sepallength']*df['petalwidth']

for name in df['class'].unique():
    df[f'label-{name}'] = df['class'].map(lambda x: 1 if x == name else 0)

In [44]:
test_frac = 0.8

p = np.random.permutation(df.index.size)

test_size = int(p.size*test_frac)
train_size = int(p.size*(1-test_frac))

idx_test = p[0 : test_size]
idx_train = p[test_size: p.size]

feature_columns = ['sepallength', 'sepalwidth', 'petallength', 'petalwidth']
label_columns = ['label-Iris-setosa', 'label-Iris-versicolor', 'label-Iris-virginica']

xTest = np.array(df.iloc[idx_test][feature_columns])
yTest = np.array(df.iloc[idx_test][label_columns])

xTrain = np.array(df.iloc[idx_train][feature_columns])
yTrain = np.array(df.iloc[idx_train][label_columns])

In [45]:
nIn = 4
nOut = 3

learning_rate = 1e-2
batch_size = 15
max_iter = 10000

model = Perceptron(nIn, nOut)

print('untrained loss: ', model.loss(xTest, yTest).round(4))

model.train(
    xTrain,
    yTrain,
    learning_rate,
    batch_size,
    max_iter
)

print('trained loss: ', model.loss(xTest, yTest).round(4))

untrained loss:  0.5818
trained loss:  0.0223


In [46]:
for x,y in zip(xTest, yTest):
    print(model.predict(x).round(2), y)

[1. 0. 0.] [1 0 0]
[0.99 0.01 0.  ] [1 0 0]
[0.   0.71 0.29] [0 1 0]
[0.   0.98 0.02] [0 1 0]
[0. 0. 1.] [0 0 1]
[0.   0.83 0.17] [0 1 0]
[0.01 0.99 0.  ] [0 1 0]
[0. 1. 0.] [0 1 0]
[0.   0.99 0.01] [0 1 0]
[0. 1. 0.] [0 1 0]
[1. 0. 0.] [1 0 0]
[1. 0. 0.] [1 0 0]
[0. 1. 0.] [0 1 0]
[0.   0.01 0.99] [0 0 1]
[1. 0. 0.] [1 0 0]
[1. 0. 0.] [1 0 0]
[0. 0. 1.] [0 0 1]
[0.   0.01 0.99] [0 0 1]
[0. 0. 1.] [0 0 1]
[0.   0.01 0.99] [0 0 1]
[1. 0. 0.] [1 0 0]
[0.  0.1 0.9] [0 0 1]
[1. 0. 0.] [1 0 0]
[0. 1. 0.] [0 1 0]
[0.   0.18 0.82] [0 0 1]
[0. 0. 1.] [0 0 1]
[1. 0. 0.] [1 0 0]
[0. 1. 0.] [0 1 0]
[0.98 0.02 0.  ] [1 0 0]
[1. 0. 0.] [1 0 0]
[0.01 0.98 0.01] [0 1 0]
[0.   0.99 0.01] [0 1 0]
[0. 0. 1.] [0 0 1]
[0. 0. 1.] [0 0 1]
[0. 0. 1.] [0 0 1]
[0.   0.01 0.99] [0 0 1]
[0.   0.99 0.01] [0 1 0]
[0.98 0.02 0.  ] [1 0 0]
[1. 0. 0.] [1 0 0]
[0.04 0.96 0.  ] [0 1 0]
[0. 0. 1.] [0 0 1]
[1. 0. 0.] [1 0 0]
[0. 0. 1.] [0 0 1]
[0.99 0.01 0.  ] [1 0 0]
[0. 1. 0.] [0 1 0]
[0.98 0.02 0.  ] [1 0 0]
[0.   0.9